## Pandas e Imputación de Missings y Outliers

### 1. Pandas

Pandas es una librería de Python usada para trabajar con datasets.

Usando Pandas podemos analizar, limpiar, explorar y manipular datos de forma muy fácil.

##### ¿Qué es una librería?

Una librería es una colección de funciones y métodos que nos permiten realizar muchas acciones usando código que ya fue escrito (probablemte por alguien más).

##### ¿Cómo usar las funciones de Pandas?

Si están trabajando en Colab, Pandas viene instalado por defecto, por lo que basta con importar la librería como en la celda siguiente (noten que usamos el alias ```pd``` para referirnos a la librería):

(Si están trabajando en su propio ambiente, instálenla usando ```!pip install pandas```)

In [ ]:
import pandas as pd

##### ¿Qué estructuras de datos nos provee Pandas?

Pandas nos provee de dos estructuras de datos:
- Series: Una Serie es un conjunto de datos unidimensional (de cualquier tipo), cada uno asociado a un índice único. Es como una columna dentro de una tabla. 

- DataFrame: Un DataFrame es un conjunto datos bidimensional (de cualquier tipo). Es como una tabla con filas y columnas (o un conjunto de varias Series).

Podemos crear Series y DataFrames a partir de Listas o Diccionarios:

In [ ]:
letras = ['a', 'b', 'c', 'd', 'e']
numeros = [1, 2, 3, 4, 5]
my_1d_dict = {letra: numero for letra, numero in zip(letras, numeros)}

print('Nuestro diccionario:', my_1d_dict)

series_from_list = pd.Series(letras)
print("Nuestra serie a partir de una lista:\n", series_from_list)

series_from_dict = pd.Series(my_1d_dict)
print("Nuestra serie a partir de un diccionario:\n", series_from_dict)

In [ ]:
letras = ['a', 'b', 'c', 'd', 'e']
numeros = [1, 2, 3, 4, 5]
my_dict = {'letras': letras, 'numeros': numeros}

print('Nuestro diccionario:', my_dict)

df_from_lists = pd.DataFrame(zip(letras, numeros), columns = ['letras', 'numeros'])
print("Nuestro DataFrame a partir de listas:\n", df_from_lists)

df_from_dict = pd.DataFrame.from_dict(my_dict, orient='columns')
print("Nuestro DataFrame a partir de un diccionario:\n", df_from_dict)

##### Importar datos

Pandas nos permite importar fácilmente distintos tipos de archivos de datos como excel, csv, parquet, json, etc. También puede leer datos directamente desde un servidor SQL.

Para esta ayudantía vamos a trabajar con un dataset de tipos de cambio de distintas monedas respecto al dólar. Los datos originales los pueden en contrar en https://si3.bcentral.cl/Siete/ES/Siete/Cuadro/CAP_TIPO_CAMBIO/MN_TIPO_CAMBIO4/TCB_510_PARIDADES/TCB_510. Se les hicieron algunas modificaciones para fines pedagógicos.

Vamos a leer el archivo directamente desde Drive en formato .csv:

In [ ]:
url = "https://drive.google.com/uc?id=18CFi-KxkVe0VRrg4awCXE0585GHRvvx4"
df = pd.read_csv(url)
df.head() # Ocupamos el método head para ver las primeras lineas del DF

##### Primera mirada al dataset

Pandas tiene algunos métodos y propiedades que nos van a ayudar de forma fácil a tener una idea rápida de con qué datos estamos trabajando:

In [ ]:
# Qué forma tienen nuestros datos?
print(df.shape) #Número de Filas, Número de Columnas

In [ ]:
# Qué columnas tiene nuestro dataset?
print(df.columns)

In [ ]:
# Qué tipo de datos corresponde a cada columna y cuántos valores no vacíos hay?
df.info()

In [ ]:
# Estadísticas descriptivas de las columnas numéricas
df.describe()

##### Acceso a datos

Para acceder a ciertos datos específicos de un DataFrame tenemos dos opciones para hacerlo:

- Acceso por etiqueta (o nombre de la columna/fila): Para esto ocupamos el método ```.loc[x,y]``` de nuestro DataFrame, donde ```x``` va a representar los nombres de las filas a las que queremos acceder (el nombre del índice asociado a cada fila) e ```y``` va a representar los nombres de las columnas a las que queremos acceder.

- Acceso por posición: Para esto ocupamos el método ```.iloc[x,y]``` de nuestro DataFrame, donde ```x``` va a representar la posición (recuerden que parten desde 0!) de las filas a las que queremos acceder e ```y``` va a representar laa posición de las columnas a las que queremos acceder.

In [ ]:
# Ejemplo: acceder a la cuarta fila de la columna CLP/USD
# Por etiqueta
print(df.loc[3, 'CLP/USD'])
print(df.iloc[3, 3])

In [ ]:
# Ejemplo: Acceder a las primeras 5 filas de la columna ARS/USD
print(df.loc[:5, 'ARS/USD'])
print(df.iloc[:5, 1])

In [ ]:
# Si quieren acceder a una columna completa, pueden simplemente usar el nombre de la columna entre [], sin usar ningún método
print(df['EUR/USD'])

In [ ]:
# Cuando quieran acceder a más de una columna, deben entregar entre los [] una lista con los nombres
print(df[['CLP/USD', 'PEN/USD']])

##### Filtros

Pandas nos permite acceder a columnas o filas basado en filtros que queramos aplicar. 

Para esto tenemos que usar los operadores lógicos que aprendimos en la primera ayudantía, con la salvedad de que ```and``` se debe reemplazar por ```&``` y ```or``` por ```|```

In [ ]:
# Acceder a todas las filas en que el tipo de cambio CLP/USD fue menor a 470
df[df['CLP/USD'] < 470]

In [ ]:
# Acceder a todas las filas en que el tipo de cambio CLP/USD fue menor a 500 y el tipo de cambio ARS/USD fue menor a 4
df[(df['CLP/USD'] < 500) & (df['ARS/USD'] < 4)]

In [ ]:
# Acceder a todas las filas en que el tipo de cambio CLP/USD fue menor a 480 o el tipo de cambio ARS/USD fue mayor a 500
df[(df['CLP/USD'] < 480) | (df['ARS/USD'] > 500)]

##### Creación de columnas

Para crear una columna nueva en un DataFrame ya existente, simplemente se debe asignar un valor único o una Serie del mismo largo al nuevo nombre de columna:

```
df['new_column'] = valor_unico
df['new_column'] = serie_del_mismo_largo
```

In [ ]:
df['Fuente'] = 'BCCH'
print(df.head())

##### Trabajo con fechas

Pandas nos provee de herramientas para manipular fechas pero primero debemos decirle que una columna representa fechas.

Para esto usamos el método ```pd.to_datetime()``` al que le debemos entregar la columna que contiene fechas y el formato de esas fechas

In [ ]:
df['Fecha'] = pd.to_datetime(df['Fecha'], format = '%d-%m-%y')
df.head()

In [ ]:
df.info()

Ahora que ya tenemos nuestra columna con formato fecha, podemos ocupar los métodos de Pandas para trabajar con fechas usando el accesor ```.dt```. Por ejemplo, usaremos el método ```.dt.year``` para extraer el año de cada fecha:

In [ ]:
df['Año'] = df['Fecha'].dt.year
print(df.head())

Además, podemos aprovechar que creamos esta columna para ver cuántos datos por año tenemos usando el método ```value_counts()``` (esto lo pueden usar con cualquier columna, no es necesario que sea fecha):

In [ ]:
print(df['Año'].value_counts())

##### Agrupación de datos

Si tenemos distintos grupos de datos sobre los que queremos trabajar en un mismo DataFrame, podemos ocupar el método ```groupby()``` para trabajar sobre cada grupo de datos por separado.

In [ ]:
# Cálculo de promedio anual de cada tipo de cambio
df.groupby(by='Año').mean()

### 2. Imputación de Missings y Outliers

Antes de entrar de lleno en la imputación de missings y outliers, veamos gráficamente nuestras series:

In [ ]:
for column in df.columns:
    if "/USD" in column:
        df.plot(x = 'Fecha', y=column, title=f"Tipo de cambio {column}")

Vemos que al parecer solo hay missing values en la serie ARS/USD y que la serie CLP/USD tiene algunos outliers.

Veamos ahora si es que tenemos más de un dato por fecha:

In [ ]:
df.Fecha.value_counts()

Tenemos 2 datos para Febrero de 2024! Vamos a tener que trabajar eso también.

##### Imputación de Missings

Partamos trabajando sobre los missings:

Primero, veamos cuántos son y en qué fechas son:

In [ ]:
df.isna().sum() # Aquí ocupamos el método isna() que nos devuelve valores booleanos indicando los datos nulos y
                # luego los sumamos para obtener el total por columna

In [ ]:
df[df['ARS/USD'].isna()]

Solo tenemos 3 datos faltantes, entre junio y agosto de 2014.

Para hacernos cargo de esto tenemos dos grandes opciones:

- Eliminar las filas que tienen valores faltantes, lo que no es muy conveniente en el contexto de series de tiempo, pero sí puede ser muy útil cuando estén trabajando con otro tipo de datos.

- Imputar los valores faltantes usando alguna técnica de las muchas que hay: forward filling, backward filling, usar el promedio (estas 3 primeras opciones las pueden lograr usando el método ```fillna()```), interpolar, hacer un forecast para esos datos usando los comportamientos estacionales de nuestra serie, etc.

Por ahora, los vamos a llenar con una interpolación lineal simple:

In [ ]:
df['ARS/USD'] = df['ARS/USD'].interpolate()
df.plot(x='Fecha', y='ARS/USD', title = "Tipo de Cambio ARS/USD Interpolado")

In [ ]:
df[(df.Fecha >= '2014-05-01') & (df.Fecha <= '2014-09-01')]

##### Eliminación de duplicados

Vamos a tratar ahora el problema de los datos duplicados para Febrero de 2024, veamos qué datos tenemos para esa fecha:

In [ ]:
df[df['Fecha'] == "2024-02-01"]

Al parecer solo fue un error en que se ingresó dos veces los mismos datos, podría haber sido peor si tuviésemos distintos valores para una misma fecha.

Para arreglar esto usamos el método ```drop_duplicates()```. A él le podemos entregar el argumento ```keep = ...``` para decirle que mantenga el primero de nuestros datos duplicados (```"first"```), el último (```"last"```) o ninguno (```False```). Vamos a quedarnos con el primero:

In [ ]:
df = df.drop_duplicates(keep = 'first')
df[df['Fecha'] == "2024-02-01"]

##### Imputación de Outliers

El análisis gráfico ya nos ayudó a identificar que solo en la serie CLP/USD tenemos outliers. Veamos más de cerca esta serie:

In [ ]:
df['CLP/USD'].describe()

In [ ]:
df['CLP/USD'].sort_values().head()

In [ ]:
df['CLP/USD'].sort_values(ascending=False).head()

In [ ]:
df.boxplot(column=["CLP/USD"])

Las estadísticas descriptivas nos muestran que tenemos un mínimo y un máximo muy alto para esta serie. Además, al mirar los datos ordenados de mayor a menor, de menor a mayor y el boxplot de la serie nos damos cuenta que solo son el mínimo y el máximo los valores atípicos.

Para poder imputarlos, los reemplazaremos por un missing value y los trataremos igual que como tratamos los valores faltantes de la serie ARS/USD (por interpolación lineal):

In [ ]:
# Primero identifiquemos las filas en que tenemos que imputar:
df[df['CLP/USD'].isin([df['CLP/USD'].min(), df['CLP/USD'].max()])]

In [ ]:
import numpy as np
df.loc[[177,193], 'CLP/USD'] = np.nan
df.loc[[177,193], 'CLP/USD']

In [ ]:
df['CLP/USD'] = df['CLP/USD'].interpolate()
df.plot(x='Fecha', y='CLP/USD', title = "Tipo de Cambio CLP/USD Interpolado")

In [ ]:
df.loc[[177,193], 'CLP/USD']